In [82]:
from correctingagent.world import world
from correctingagent.agents import agents
from correctingagent.agents.teacher import TeacherAgent
from correctingagent.agents.agents import CorrectingAgent
import os
import pandas as pd
import pickle
from correctingagent.experiments.evaluation import test_colour_model, ResultsFile
import configparser
import logging
from collections import defaultdict
from correctingagent.models import prob_model
import sqlalchemy
from correctingagent.util.util import config_location, get_config, get_neural_config, get_kde_config
from pgmpy.models import FactorGraph
from pgmpy.factors.discrete import TabularCPD, DiscreteFactor
from pgmpy.inference import VariableElimination
from KDEpy import NaiveKDE
from pgmpy.factors.continuous import ContinuousFactor
from correctingagent.models.prob_model import KDEColourModel
import nltk
from nltk.sem import Valuation, Model
import numpy as np
from collections import defaultdict
from pgmpy.factors import factor_product

In [2]:
# def get_agent(config):
#     if config['agent'] == 'agents.CorrectingAgent':
#         agent = PGMCorrectingAgent
#     elif config['agent'] == 'agents.RandomAgent':
#         agent = agents.RandomAgent
#     elif config['agent'] == 'agents.NeuralCorrectingAgent':
#         agent = agents.NeuralCorrectingAgent
#     elif config['agent'] == 'agents.PerfectColoursAgent':
#         agent = agents.PerfectColoursAgent
#     elif config['agent'] == 'agents.NoLanguageAgent':
#         agent = agents.NoLanguageAgent
#     else:
#         raise ValueError('invalid agent name')
#     return agent


# class PGMCorrectingAgent(CorrectingAgent):
#     pass

In [3]:
# config = get_config()
# data_location = config['data_location']
# db_location = config['db_location']
# handler = logging.StreamHandler()
# logger = logging.getLogger('dialogue')
# logger.setLevel(logging.INFO)
# logger.addHandler(handler)

# def run_experiment(config_name='DEFAULT', debug=False, neural_config='DEFAULT'):

#     if debug:
#         agent_logger.setLevel(logging.DEBUG)

#     config = configparser.ConfigParser()
#     config_file = os.path.join(config_location, 'experiments.ini')
#     config.read(config_file)
#     config = config[config_name]
#     problem_name = config['scenario_suite']
#     threshold = config.getfloat('threshold')
#     update_negative = config.getboolean('update_negative')
#     Agent = get_agent(config)
#     vis = config.getboolean('visualise')
#     update_once = config.getboolean('update_once')
#     colour_model_type = config['colour_model_type']
#     no_correction_update = config.getboolean('no_correction_update')



#     if debug and not 'Random' in config['agent']:
#         debugger = Debug(config)


#     total_reward = 0
#     problem_dir = os.path.join(data_location, problem_name)
#     problems = os.listdir(problem_dir)
#     w = world.PDDLWorld('blocks-domain.pddl', os.path.join(problem_dir, problems[0]))
#     teacher = TeacherAgent()
#     if Agent in [agents.NeuralCorrectingAgent]:
#         config_dict = get_neural_config(neural_config)
#         agent = Agent(w, teacher=teacher, **config_dict)
#     elif Agent in [agents.CorrectingAgent, agents.NoLanguageAgent]:
#         if colour_model_type == 'kde':
#             if neural_config is None:
#                 neural_config = 'DEFAULT'
#             model_config = get_kde_config(neural_config)
#         else:
#             model_config = {}
#         agent = Agent(w, teacher=teacher, threshold=threshold, update_negative=update_negative, update_once=update_once, colour_model_type=colour_model_type, model_config=model_config)
#     else:
#         agent = Agent(w, teacher=teacher, threshold=threshold)


#     print('Results for {}\n'.format(problem_name))
#     for problem in problems:
#         w = world.PDDLWorld('blocks-domain.pddl', os.path.join(problem_dir, problem))
#         agent.new_world(w)
#         while not w.test_success():
#             plan = agent.plan()
#             for a, args in plan:
#                 if a == 'reach-goal':
#                     break
#                 w.update(a, args)
#                 if vis:
#                     w.draw()
#                 correction = agent.teacher.correction(w)
#                 if correction:
#                     logger.info("T: " + correction)
#                     agent.get_correction(correction, a, args)
#                     if vis:
#                         w.draw()
#                     break
#                 elif no_correction_update:
#                     agent.no_correction(a, args)
#         if debug and not 'Random' in config['agent']:
#             debugger.cm_confusion(agent)
#             debugger.update_cm_params(agent)



#         total_reward += w.reward
#         print('{} reward: {}'.format(problem, w.reward))

#         print('{} reward: {}\n'.format(problem, w.reward))
#         print('{} cumulative reward: {}\n'.format(problem, total_reward))


#     print('total reward: {}\n'.format(total_reward))

#     return results_file.name

In [4]:
# run_experiment(config_name='simple_kde', neural_config='fixed_bw_0.2')

In [140]:
cm = KDEColourModel('blue')
cm2 = KDEColourModel('red')
# cm.update([1,0,0], 0.9)
# cm.update([0.7, 0.2, 0], 0.8)

# cm2.update([0,0,1], 0.9)
# cm2.update([0.5, 0.1, 0.9], 0.8)
KDE_colour_model = lambda r,g,b,c: cm.p(c, [r,g,b])
KDE_colour_model2 = lambda r,g,b,c:cm2.p(c, [r,g,b])

blue_cm = ContinuousFactor(['r2', 'g2', 'b2', 'blue'], KDE_colour_model)
red_cm = ContinuousFactor(['r1', 'g1', 'b1', 'red'], KDE_colour_model2)

In [7]:
print(blue_cm.assignment(1,0,0, 1))
print(red_cm.assignment(1,0,0,1))


0.9555148089163196
0.0020056121625727052


In [8]:
def create_rules(colour1, colour2):
    r1 = 'all x.({}(x) -> exists y. ({}(y) & on(x,y)))'.format(colour1, colour2)
    r2 = 'all y.({}(y) -> exists x. ({}(x) & on(x,y)))'.format(colour2, colour1)
    return r1, r2

def evaluate_rule(colour1, value1, colour2, value2, rule):
    c1_set = set()
    c2_set = set()
    if value1 == 1:
        c1_set.add('o1')
    if value2 == 1:
        c2_set.add('o2')
    v = [(colour1, c1_set), (colour2, c2_set),
        ('on', set([('o1', 'o2')]))]
    
    val = Valuation(v)
    dom = val.domain
    m = Model(dom, val)
    g = nltk.sem.Assignment(dom)
    return m.evaluate(rule, g)

In [9]:
r1, r2 = create_rules('red', 'blue')

In [10]:
evaluate_rule('red', 1, 'blue', 0, r1)

False

In [103]:
def generate_CPD(rule, c1, c2):
    cpd_line_corr0 = []
    cpd_line_corr1 = []
    
    cpd = np.zeros((2,2))
    for i in range(2):
        for j in range(2):
            for r in range(2):
                result = r * (1-int(evaluate_rule(c1, i, c2, j, rule)))
                cpd_line_corr1.append(result)
                cpd_line_corr0.append(1-result)
    
    return [cpd_line_corr0, cpd_line_corr1]

def or_CPD():
    cpd_line1 = []
    cpd_line0 = []
    cpd = np.zeros((2,2))
    for i in range(2):
        for j in range(2):
            cpd_line1.append(int(i or j))
            cpd_line0.append(1-int(i or j))
    return [cpd_line0, cpd_line1]

In [110]:
table = generate_CPD(r1, 'red', 'blue')
table
table2 = generate_CPD(r2, 'red', 'blue')
table2
correction_table = or_CPD()
correction_table

[[1, 0, 0, 0], [0, 1, 1, 1]]

In [141]:
violated_r1 = TabularCPD('violated_r1', 2, table, evidence=['red', 'blue', 'rule1'], evidence_card = [2,2,2])
violated_r1_factor = violated_r1.to_factor()
violated_r2 = TabularCPD('violated_r2', 2, table2, evidence=['red', 'blue', 'rule2'], evidence_card = [2,2,2])
violated_r2_factor = violated_r2.to_factor()
correction = TabularCPD('corr', 2, correction_table, evidence=['violated_r1', 'violated_r2'], evidence_card=[2,2])
correction_factor = correction.to_factor()

In [14]:
print(corr)

+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| red    | red_0  | red_0  | red_0  | red_0  | red_1  | red_1  | red_1  | red_1  |
+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| blue   | blue_0 | blue_0 | blue_1 | blue_1 | blue_0 | blue_0 | blue_1 | blue_1 |
+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| r      | r_0    | r_1    | r_0    | r_1    | r_0    | r_1    | r_0    | r_1    |
+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| corr_0 | 1.0    | 1.0    | 1.0    | 1.0    | 1.0    | 0.0    | 1.0    | 1.0    |
+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| corr_1 | 0.0    | 0.0    | 0.0    | 0.0    | 0.0    | 1.0    | 0.0    | 0.0    |
+--------+--------+--------+--------+--------+--------+--------+--------+--------+


In [155]:
G = FactorGraph()
G.add_nodes_from(['corr', 'violated_r1', 'violated_r2', 
                  'r1', 'g1', 'b1', 'r2', 'g2', 'b2', 
                  'red', 'blue', 'rule1', 'rule2'])
G.add_factors(correction_factor)
G.add_factors(violated_r1_factor)
G.add_factors(violated_r2_factor)
#red_cm.variables = ['r1', 'g1', 'b1', 'red']
red_cm.cardinality = [1, 1, 1, 2]
G.add_factors(red_cm)
#blue_cm.variables = ['r2', 'g2', 'b2', 'blue']
blue_cm.cardinality = [1,1,1,2]
G.add_factors(blue_cm)
#G.add_nodes_from([corr_factor, red_cm, blue_cm])
G.add_edges_from([('corr', correction_factor), ('violated_r2', correction_factor), ('violated_r1', correction_factor)])

G.add_edges_from([('red', violated_r1_factor), ('blue', violated_r1_factor), 
                  ('rule1', violated_r1_factor), ('violated_r1', violated_r1_factor)])

G.add_edges_from([('violated_r2', violated_r2_factor), ('red', violated_r2_factor),
                 ('blue', violated_r2_factor), ('rule2', violated_r2_factor)])

G.add_edges_from([('r1', red_cm), ('g1', red_cm), ('b1', red_cm), ('red', red_cm)])
G.add_edges_from([('r2', blue_cm), ('g2', blue_cm), ('b2', blue_cm), ('blue', blue_cm)])

In [156]:
G.nodes()

['corr',
 'violated_r1',
 'violated_r2',
 'r1',
 'g1',
 'b1',
 'r2',
 'g2',
 'b2',
 'red',
 'blue',
 'rule1',
 'rule2',
 <DiscreteFactor representing phi(corr:2, violated_r1:2, violated_r2:2) at 0x7f70149c6860>,
 <DiscreteFactor representing phi(violated_r1:2, red:2, blue:2, rule1:2) at 0x7f7014955ef0>,
 <DiscreteFactor representing phi(violated_r2:2, red:2, blue:2, rule2:2) at 0x7f70149c6898>,

In [157]:
inference = VariableElimination(G)

In [159]:
q = inference.query(variables=['rule1', 'rule2', 'red', 'blue'], 
                evidence={'r1':0, 'g1':0.5, 'b1':0.8, 'corr':1, 'r2':1, 'g2':0.8, 'b2':0.4}, 
                elimination_order=['violated_r1', 'violated_r2'])
print(q['red'])
print(q['blue'])
print(q['rule1'])
print(q['rule2'])

0
['violated_r1', 'red', 'blue', 'rule1']
['violated_r1', 'violated_r2']
1
['red', 'blue', 'rule1', 'violated_r2']
['violated_r2', 'red', 'blue', 'rule2']
['red']
['blue']
['red', 'blue', 'rule1', 'rule2']
['red']
['blue']
['red', 'blue', 'rule1', 'rule2']
['red']
['blue']
['red', 'blue', 'rule1', 'rule2']
['red']
['blue']
['red', 'blue', 'rule1', 'rule2']
+-------+------------+
| red   |   phi(red) |
+=======+============+
| red_0 |     0.5000 |
+-------+------------+
| red_1 |     0.5000 |
+-------+------------+
+--------+-------------+
| blue   |   phi(blue) |
+========+=============+
| blue_0 |      0.5000 |
+--------+-------------+
| blue_1 |      0.5000 |
+--------+-------------+
+---------+--------------+
| rule1   |   phi(rule1) |
+=========+==============+
| rule1_0 |       0.2500 |
+---------+--------------+
| rule1_1 |       0.7500 |
+---------+--------------+
+---------+--------------+
| rule2   |   phi(rule2) |
+=========+==============+
| rule2_0 |       0.2500 |
+-------